In [4]:
bucket = 'cmc8dm-pa5-2'
prefix = 'sagemaker/xgboost_credit_risk'

# Define IAM role
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

role = get_execution_role()

In [5]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2019-11-13 20:07:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls.1’

default of credit c 100%[===================>]   5.28M  8.88MB/s    in 0.6s    

2019-11-13 20:07:20 (8.88 MB/s) - ‘default of credit card clients.xls.1’ saved [5539328/5539328]



In [6]:
dataset = pd.read_excel('default of credit card clients.xls',index_col=0)
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 15)
dataset

,X1,X2,X3,X4,X5,X6,X7,...,X18,X19,X20,X21,X22,X23,Y
ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,...,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,...,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,150000,1,3,2,43,-1,-1,...,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,...,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,...,85900,3409,1178,1926,52964,1804,1
30000,50000,1,2,1,46,0,0,...,2078,1800,1430,1000,1000,1000,1


In [7]:
dataset = dataset.drop('ID')
dataset = pd.concat([dataset['Y'], dataset.drop(['Y'], axis=1)], axis=1)

In [8]:
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [9]:
s3_input_train = boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_validation = boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [10]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix),
content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation'.format(bucket, prefix),
content_type='csv')

In [ ]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1,
                        objective='binary:logistic',
                        num_round=25)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [13]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1,
                        objective='binary:logistic',
                        num_round=25)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2019-11-13 21:07:13 Starting - Starting the training job...
2019-11-13 21:07:25 Starting - Launching requested ML instances.........
2019-11-13 21:08:59 Starting - Preparing the instances for training......
2019-11-13 21:10:01 Downloading - Downloading input data...
2019-11-13 21:10:41 Training - Training image download completed. Training in progress.
2019-11-13 21:10:41 Uploading - Uploading generated training model.Arguments: train
[2019-11-13:21:10:36:INFO] Running standalone xgboost training.
[2019-11-13:21:10:36:INFO] File size need to be processed in the node: 2.32mb. Available memory size in the node: 8614.02mb
[2019-11-13:21:10:36:INFO] Determined delimiter of CSV input is ','
[21:10:36] S3DistributionType set as FullyReplicated
[21:10:36] 21000x23 matrix with 483000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2019-11-13:21:10:36:INFO] Determined delimiter of CSV input is ','
[21:10:36] S3DistributionType set as FullyReplicated
[21:10:36

In [15]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------------------------------------------------------------------------------------!

In [16]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])
predictions

/home/ec2-user/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


array([0.59283876, 0.1414548 , 0.31579986, ..., 0.41416201, 0.11323805,
       0.11232713])

In [41]:
cm = pd.crosstab(index=test_data['Y'], columns=np.round(predictions), rownames=['Observed'], colnames=['Predicted'])
tn = float(cm.iloc[0,0]); fn = float(cm.iloc[1,0]); tp = float(cm.iloc[1,1]); fp = float(cm.iloc[0,1]); p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<20}{1:<20}{2:>10}".format("Predicted", "No Default Credit", "Default Credit"))
print("Observed")
print("{0:<20}{1:<2.0f}% ({2:<}){3:>10.0f}% ({4:<})".format("No Default Credit", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<20}{1:<1.0f}% ({2:<}){3:>11.0f}% ({4:<}) \n".format("Default Credit", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 82.2%

Predicted           No Default Credit   Default Credit
Observed
No Default Credit   84% (2219.0)        33% (121.0)
Default Credit      16% (414.0)         67% (246.0) 

